In [12]:
#Question 9: Distance Matrix Calculation
    
import pandas as pd
import numpy as np

def calculate_distance_matrix(file_path):
    # Read the dataset
    df = pd.read_csv("C:/Users/sahit/Downloads/dataset-2.csv")
    
    # Get all unique locations
    unique_ids = np.unique(df[['id_start', 'id_end']].values)
    
    # Initialize a DataFrame for the distance matrix
    distance_matrix = pd.DataFrame(data=np.inf, index=unique_ids, columns=unique_ids)
    
    # Set diagonal to 0 (distance from a location to itself)
    np.fill_diagonal(distance_matrix.values, 0)
    
    # Fill the matrix with given distances
    for _, row in df.iterrows():
        id_start, id_end, distance = row['id_start'], row['id_end'], row['distance']
        distance_matrix.at[id_start, id_end] = distance
        distance_matrix.at[id_end, id_start] = distance  # Ensure symmetry
    
    # Floyd-Warshall algorithm to compute the shortest paths for cumulative distances
    for k in unique_ids:
        for i in unique_ids:
            for j in unique_ids:
                distance_matrix.at[i, j] = min(distance_matrix.at[i, j], distance_matrix.at[i, k] + distance_matrix.at[k, j])
    
    return distance_matrix

# Usage example
file_path = "C:/Users/sahit/Downloads/dataset-2.csv"  # Replace with your actual file path
distance_matrix = calculate_distance_matrix(file_path)
print(distance_matrix)


         1001400  1001402  1001404  1001406  1001408  1001410  1001412  \
1001400      0.0      9.7     29.9     45.9     67.6     78.7     94.3   
1001402      9.7      0.0     20.2     36.2     57.9     69.0     84.6   
1001404     29.9     20.2      0.0     16.0     37.7     48.8     64.4   
1001406     45.9     36.2     16.0      0.0     21.7     32.8     48.4   
1001408     67.6     57.9     37.7     21.7      0.0     11.1     26.7   
1001410     78.7     69.0     48.8     32.8     11.1      0.0     15.6   
1001412     94.3     84.6     64.4     48.4     26.7     15.6      0.0   
1001414    112.5    102.8     82.6     66.6     44.9     33.8     18.2   
1001416    125.7    116.0     95.8     79.8     58.1     47.0     31.4   
1001418    139.3    129.6    109.4     93.4     71.7     60.6     45.0   
1001420    152.2    142.5    122.3    106.3     84.6     73.5     57.9   
1001422    161.8    152.1    131.9    115.9     94.2     83.1     67.5   
1001424    173.2    163.5    143.3    

In [13]:
#Question 10: Unroll Distance Matrix
    
import pandas as pd

def unroll_distance_matrix(distance_matrix):
    # Initialize a list to store the results
    data = []
    
    # Loop over the distance matrix to unroll it into id_start, id_end, distance
    for id_start in distance_matrix.index:
        for id_end in distance_matrix.columns:
            if id_start != id_end:  # Skip diagonal elements where id_start == id_end
                distance = distance_matrix.at[id_start, id_end]
                data.append([id_start, id_end, distance])
    
    # Create a DataFrame from the unrolled data
    unrolled_df = pd.DataFrame(data, columns=['id_start', 'id_end', 'distance'])
    
    return unrolled_df

# Example usage:
unrolled_df = unroll_distance_matrix(distance_matrix)
print(unrolled_df)

      id_start   id_end  distance
0      1001400  1001402       9.7
1      1001400  1001404      29.9
2      1001400  1001406      45.9
3      1001400  1001408      67.6
4      1001400  1001410      78.7
...        ...      ...       ...
1801   1004356  1001470     159.8
1802   1004356  1001472     175.8
1803   1004356  1001488       4.0
1804   1004356  1004354       2.0
1805   1004356  1004355       4.0

[1806 rows x 3 columns]


In [14]:
# Question 11: Finding IDs within Percentage Threshold
    
import pandas as pd

def find_ids_within_ten_percentage_threshold(df, reference_value):
    # Step 1: Filter rows where id_start matches the reference value
    reference_rows = df[df['id_start'] == reference_value]
    
    # Step 2: Calculate the average distance for the reference value
    reference_avg = reference_rows['distance'].mean()
    
    # Step 3: Calculate the 10% threshold
    lower_bound = reference_avg * 0.9  # 10% below the average
    upper_bound = reference_avg * 1.1  # 10% above the average
    
    # Step 4: Identify id_start values whose average distances are within the threshold
    valid_ids = []
    for id_value in df['id_start'].unique():
        avg_distance = df[df['id_start'] == id_value]['distance'].mean()
        if lower_bound <= avg_distance <= upper_bound:
            valid_ids.append(id_value)
    
    # Step 5: Return the sorted list of valid ids
    return sorted(valid_ids)

# Example usage:
unrolled_df = unroll_distance_matrix(distance_matrix)  # From the previous question
reference_value = 1001400  # Replace with the reference id_start value you want to check
result_ids = find_ids_within_ten_percentage_threshold(unrolled_df, reference_value)
print(result_ids)


[1001400, 1001402]


In [16]:
#Question 12: Calculate Toll Rate
    
import pandas as pd

def calculate_toll_rate(df):
    # Define rate coefficients for each vehicle type
    rates = {
        'moto': 0.8,
        'car': 1.2,
        'rv': 1.5,
        'bus': 2.2,
        'truck': 3.6
    }
    
    # For each vehicle type, calculate the toll rate by multiplying distance with the rate coefficient
    for vehicle_type, rate in rates.items():
        df[vehicle_type] = df['distance'] * rate
    
    # Drop the distance column from the DataFrame
    df = df.drop(columns=['distance'])
    
    return df

# Example usage:
unrolled_df = unroll_distance_matrix(distance_matrix)  # From the previous question
toll_rate_df = calculate_toll_rate(unrolled_df)
print(toll_rate_df)


      id_start   id_end    moto     car      rv     bus   truck
0      1001400  1001402    7.76   11.64   14.55   21.34   34.92
1      1001400  1001404   23.92   35.88   44.85   65.78  107.64
2      1001400  1001406   36.72   55.08   68.85  100.98  165.24
3      1001400  1001408   54.08   81.12  101.40  148.72  243.36
4      1001400  1001410   62.96   94.44  118.05  173.14  283.32
...        ...      ...     ...     ...     ...     ...     ...
1801   1004356  1001470  127.84  191.76  239.70  351.56  575.28
1802   1004356  1001472  140.64  210.96  263.70  386.76  632.88
1803   1004356  1001488    3.20    4.80    6.00    8.80   14.40
1804   1004356  1004354    1.60    2.40    3.00    4.40    7.20
1805   1004356  1004355    3.20    4.80    6.00    8.80   14.40

[1806 rows x 7 columns]


In [17]:
#Question 13: Calculate Time-Based Toll Rates

def calculate_time_based_toll_rates(df):
    import pandas as pd
    from datetime import time
    
    # Initialize an empty DataFrame
    time_based_df = pd.DataFrame(columns=[
        'id_start', 'id_end', 'start_day', 'start_time', 'end_day', 'end_time',
        'moto', 'car', 'rv', 'bus', 'truck'
    ])
    
    # List of days of the week
    days_of_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    
    # Time intervals for weekdays
    weekday_intervals = [
        (time(0, 0), time(10, 0), 0.8),
        (time(10, 0), time(18, 0), 1.2),
        (time(18, 0), time(23, 59, 59), 0.8)
    ]
    
    # Iterate over each row in the original DataFrame
    for _, row in df.iterrows():
        # Iterate over each day of the week
        for day in days_of_week:
            # If it's a weekday, apply the 3 time intervals
            if day in ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']:
                for start_time, end_time, discount in weekday_intervals:
                    # Calculate rates with discount
                    moto_rate = row['moto'] * discount
                    car_rate = row['car'] * discount
                    rv_rate = row['rv'] * discount
                    bus_rate = row['bus'] * discount
                    truck_rate = row['truck'] * discount
                    
                    # Create new row for the interval
                    new_row = pd.DataFrame([{
                        'id_start': row['id_start'],
                        'id_end': row['id_end'],
                        'start_day': day,
                        'start_time': start_time,
                        'end_day': day,
                        'end_time': end_time,
                        'moto': moto_rate,
                        'car': car_rate,
                        'rv': rv_rate,
                        'bus': bus_rate,
                        'truck': truck_rate
                    }])
                    
                    # Concatenate the new row
                    time_based_df = pd.concat([time_based_df, new_row], ignore_index=True)
            
            # For weekends, apply a constant discount
            elif day in ['Saturday', 'Sunday']:
                discount = 0.7
                start_time = time(0, 0)
                end_time = time(23, 59, 59)
                
                # Calculate rates with weekend discount
                moto_rate = row['moto'] * discount
                car_rate = row['car'] * discount
                rv_rate = row['rv'] * discount
                bus_rate = row['bus'] * discount
                truck_rate = row['truck'] * discount
                
                # Create a new row for the weekend day
                new_row = pd.DataFrame([{
                    'id_start': row['id_start'],
                    'id_end': row['id_end'],
                    'start_day': day,
                    'start_time': start_time,
                    'end_day': day,
                    'end_time': end_time,
                    'moto': moto_rate,
                    'car': car_rate,
                    'rv': rv_rate,
                    'bus': bus_rate,
                    'truck': truck_rate
                }])
                
                # Concatenate the new row
                time_based_df = pd.concat([time_based_df, new_row], ignore_index=True)
    
    return time_based_df
toll_rate_df = calculate_toll_rate(unrolled_df)  # From Question 12
time_based_toll_df = calculate_time_based_toll_rates(toll_rate_df)
print(time_based_toll_df)

        id_start     id_end start_day start_time   end_day  end_time   moto  \
0      1001400.0  1001402.0    Monday   00:00:00    Monday  10:00:00  6.208   
1      1001400.0  1001402.0    Monday   10:00:00    Monday  18:00:00  9.312   
2      1001400.0  1001402.0    Monday   18:00:00    Monday  23:59:59  6.208   
3      1001400.0  1001402.0   Tuesday   00:00:00   Tuesday  10:00:00  6.208   
4      1001400.0  1001402.0   Tuesday   10:00:00   Tuesday  18:00:00  9.312   
...          ...        ...       ...        ...       ...       ...    ...   
30697  1004356.0  1004355.0    Friday   00:00:00    Friday  10:00:00  2.560   
30698  1004356.0  1004355.0    Friday   10:00:00    Friday  18:00:00  3.840   
30699  1004356.0  1004355.0    Friday   18:00:00    Friday  23:59:59  2.560   
30700  1004356.0  1004355.0  Saturday   00:00:00  Saturday  23:59:59  2.240   
30701  1004356.0  1004355.0    Sunday   00:00:00    Sunday  23:59:59  2.240   

          car     rv     bus   truck  
0       9.31